# 0.3 Reproject stake coordinates to Lat, Lon

**Get latitude and longitude of UTM coordinates of point measurements**

Transform approximate and exact coordinates from UTM to latitude/longitude. Add latitude/longitude coordinates as columns.

To system:
WGS84 Latitude/Longitude: "EPSG:4326"

From system:
EUREF89 UTM32: EPSG:25832
EUREF89 UTM33: EPSG:25833
EUREF89 UTM34: EPSG:25834

Input dataset is named: 'yyyy-mm-dd_stake_mb_norway_cleaned_ids.csv'

Output dataset with lat/lon coordinates is named: 'yyyy-mm-dd_stake_mb_norway_cleaned_ids_latlon.csv'

In [2]:
# Import libraries
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from pyproj import Transformer

In [3]:
# Specify filepaths and filenames.
filepath = 'Data/'
filename = '2023-08-28_stake_mb_norway_cleaned_ids.csv'
filename_latlon = '2023-08-28_stake_mb_norway_cleaned_ids_latlon.csv'

In [4]:
# Load data.
data = pd.read_csv(filepath + filename)

data

,RGIID,GLIMSID,BREID,utm_zone,utm_east_approx,utm_north_approx,altitude_approx,location_description,location_id,stake_no,...,dt_curr_year_max_date,dt_curr_year_min_date,stake_remark,flag_correction,approx_loc,approx_altitude,diff_north,diff_east,diff_altitude,diff_netto
0,RGI60-08.01258,G021737E70130N,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1012,...,14.05.2013 00:00,07.11.2013 00:00,NaN,netto,N,N,50.0,80.0,-29.0,0.000000e+00
1,RGI60-08.01258,G021737E70130N,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,...,09.05.2014 00:00,24.09.2014 00:00,NaN,NaN,N,N,38.0,53.0,-61.0,0.000000e+00
2,RGI60-08.01258,G021737E70130N,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,...,08.05.2015 00:00,23.09.2015 00:00,NaN,NaN,N,N,32.0,38.0,-56.0,0.000000e+00
3,RGI60-08.01258,G021737E70130N,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,...,23.05.2016 00:00,22.09.2016 00:00,Snødyp ut fra stakemålingene (0.65 m) virker i...,netto,N,N,32.0,38.0,-56.0,0.000000e+00
4,RGI60-08.01258,G021737E70130N,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1016,...,25.04.2017 00:00,29.09.2017 00:00,Sondert snødyp.,NaN,N,N,9.0,-52.0,-32.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,RGI60-08.02969,G006362E59989N,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,...,03.05.2012 00:00,12.10.2012 00:00,Kombinasjon med stake 15-12,NaN,N,N,-20.0,-10.0,-2.0,0.000000e+00
4167,RGI60-08.02969,G006362E59989N,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,...,29.05.2013 00:00,24.09.2013 00:00,Endring av stakelengden fra 7. august til 24. ...,NaN,N,N,-35.0,-13.0,-2.0,4.440892e-16
4168,RGI60-08.02969,G006362E59989N,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,...,22.04.2015 00:00,14.10.2015 00:00,"Stakemålt snødyp var 3.0 m, mens to sonderinge...",NaN,N,N,-6.0,-27.0,-1.0,0.000000e+00
4169,RGI60-08.02969,G006362E59989N,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,...,28.04.2016 00:00,06.10.2016 00:00,Basert på sondert snødyp.,NaN,N,N,-20.0,-15.0,-2.0,0.000000e+00


In [5]:
# For running on raw data.

# Rename columns.
#data = data.rename(columns={"utm_east3": "utm_east_approx", 
#                            "utm_north4": "utm_north_approx", 
#                            "altitude5": "altitude_approx"})

# New columns indicating if location/altitude is approximate. Fill new column with "N" for
# location/altitude is not approximate.
#data['approx_loc'] = 'N'
#data['approx_altitude'] = 'N'

# If "utm_east" values are missing, fill column "approx_loc" with "Y" indicating
# that the location is approximate.
#data.loc[data['utm_east'].isna(), 'approx_loc'] = 'Y'

# If "altitude" values are missing, fill column "approx_altitude" with "Y"
# indicating that the altitude is approximate.
#data.loc[data['altitude'].isna(), 'approx_altitude'] = 'Y'

# Where there is no exact location, fill inn approximate location based on
# "utm_east_approx" and "utm_north_approx" in columns "utm_east" and "utm_north".
# Location is now filled for every observation, with column "approx_loc" indicating
# wether location is approximate (Y) or exact (N).
#approx_locs_east = data.loc[data['approx_loc'] == 'Y', 'utm_east_approx']
#data.loc[data['approx_loc'] == 'Y', 'utm_east'] = approx_locs_east
#approx_locs_north = data.loc[data['approx_loc'] == 'Y', 'utm_north_approx']
#data.loc[data['approx_loc'] == 'Y', 'utm_north'] = approx_locs_north

# Same operation with missing altitude, fill in values from "altitude_approx"
# in column "altitude". 
#approx_alt = data.loc[data['approx_altitude'] == 'Y', 'altitude_approx']
#data.loc[data['approx_altitude'] == 'Y', 'altitude'] = approx_alt 

# Calculate difference between approximate and exact positions and altitude as
# a measure of precision/quality of approximate locations.
#data['diff_north'] = data['utm_north'] - data['utm_north_approx']
#data['diff_east'] = data['utm_east'] - data['utm_east_approx']
#data['diff_altitude'] = data['altitude'] - data['altitude_approx']

In [6]:
# Transform UTM coordinates to lat,long (WGS84)

# WGS84 Latitude/Longitude: "EPSG:4326"
# EUREF89 UTM32: EPSG:25832
# EUREF89 UTM33: EPSG:25833
# EUREF89 UTM34: EPSG:25834

#data_coords = data[['utm_zone','utm_east_approx','utm_north_approx','utm_east','utm_north']]
data.loc[:, ('lat')] = np.nan
data.loc[:, ('lon')] = np.nan
data.loc[:, ('lat_approx')] = np.nan
data.loc[:, ('lon_approx')] = np.nan

#inProj = Proj('epsg:3857')
#outProj = Proj('epsg:4326')
#x1,y1 = -11705274.6374,4826473.6922
#x2,y2 = transform(inProj,outProj,x1,y1)
#print x2,y2

# Define output projection.
outProj = "EPSG:4326"

for i in range(data.index.start, data.index.stop):
 
    # Check UTM zone of coordinates and set initial projection.
    if data.iloc[i]['utm_zone'] == 34:
        inProj = "EPSG:25834"
    elif data.iloc[i]['utm_zone'] == 33:
        inProj = "EPSG:25833"
    elif data.iloc[i]['utm_zone'] == 32:
        inProj = "EPSG:25832"
    else: 
        print('UTM zone not recognized.')
    
    # Define transformer.
    transformer = Transformer.from_crs(inProj, outProj, always_xy=True)

    # Get UTM coordinates
    x,y = data.iloc[i]['utm_east'],data.iloc[i]['utm_north']
    x_a,y_a = data.iloc[i]['utm_east_approx'],data.iloc[i]['utm_north_approx']

    # Project to lat,lon (WGS84) coordinates
    x_p,y_p = transformer.transform(x,y)
    x_ap,y_ap = transformer.transform(x_a,y_a)
    
    # Add to dataframe
    data.loc[i, ('lat')] = y_p
    data.loc[i, ('lon')] = x_p
    data.loc[i, ('lat_approx')] = y_ap
    data.loc[i, ('lon_approx')] = x_ap

In [7]:
print(data['RGIID'].unique())

['RGI60-08.01258' 'RGI60-08.02436' 'RGI60-08.02458' 'RGI60-08.01026'
 'RGI60-08.02384' 'RGI60-08.00287' 'RGI60-08.01657' 'RGI60-08.01598'
 'RGI60-08.00295' 'RGI60-08.01484' 'RGI60-08.02666' 'RGI60-08.02650'
 'RGI60-08.02017' 'RGI60-08.01126' 'RGI60-08.00434' 'RGI60-08.01186'
 'RGI60-08.01286' 'RGI60-08.01217' 'RGI60-08.00868' 'RGI60-08.00987'
 'RGI60-08.00449' 'RGI60-08.01013' 'RGI60-08.00966' 'RGI60-08.01779'
 'RGI60-08.02966' 'RGI60-08.02963' 'RGI60-08.02962' 'RGI60-08.02643'
 'RGI60-08.02916' 'RGI60-08.02918' 'RGI60-08.02920' 'RGI60-08.02969']


In [8]:
# Save dataframe with latitude and longitude 
#data.to_csv(filepath + filename_latlon, index=False, sep=';')  
